In [1]:
import pandas as pd
import math

In [2]:
table = pd.read_csv('TABLE.csv', sep =";")
table

,DEPART,ARRIVEE,DISTANCE
0,A,B,3
1,A,C,4
2,A,D,8
3,B,C,4
4,B,E,9
5,C,D,4
6,C,E,5
7,D,E,7
8,D,F,4
9,D,B,6


In [3]:
names = [item for sublist in [list(table.DEPART) , list(table.ARRIVEE)]  for item in sublist]
names = sorted(list(set(names)))

adjmatrix = pd.DataFrame(columns = names)

for m in names:
    for n in names:
        if n in list(table[table["DEPART"] == m].ARRIVEE):
            adjmatrix.loc[m,n] = float(table[(table["DEPART"] == m) & (table["ARRIVEE"] == n)].DISTANCE)
        else:
            adjmatrix.loc[m,n] = 0
            
adjmatrix     

,A,B,C,D,E,F,G
A,0,3,4,8,0,0,0
B,0,0,4,0,9,0,0
C,0,0,0,4,5,0,0
D,0,6,0,0,7,4,0
E,0,0,5,0,0,0,8
F,0,15,0,0,0,0,10
G,0,0,0,0,0,0,0


In [44]:
# PREPROCESSING: input initial pour mapper1

adlist = []
start = "A"
stop = "G"

for m in names:
    if m == start:
        sublist = [m, float(0), "unvisited",[start]]
    else:
        sublist = [m, math.inf, "unvisited", [start]]
    
    for n in names:
        if adjmatrix.loc[m, n] > 0:
            sublist.append([n, adjmatrix.loc[m, n]])  
    if len(sublist) > 4:
        adlist.append(sublist)
adlist


[['A', 0.0, 'unvisited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', inf, 'unvisited', ['A'], ['C', 4.0], ['E', 9.0]],
 ['C', inf, 'unvisited', ['A'], ['D', 4.0], ['E', 5.0]],
 ['D', inf, 'unvisited', ['A'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

In [45]:
# ITERATION 1: mapper

mapper_output = []

for l in adlist:
    new_m = []
    
    # conditions pour visiter un node: qu'il ne soit pas encore visite et que son weight soit différent de inf
    if (l[2] == "unvisited") & (l[1] != math.inf):
        l[2] = "visited"
        
        # mise à jour du path:
        if l[0] not in l[3]:
            l[3].append(l[0])
        mapper_output.append(l)
        
        for m in l[4:]:
            # nouvelles keys créées à partir des adjancy nodes du node visité
            new_node = m.copy() # on a ici node_name + node distance to last neighbor
            
            # 2nd key - mise à jour des poids associés à chaque adjency node (but = toujours avoir distance from start)
            new_node[1] = new_node[1] + l[1]
            
            # 3ème key - status
            new_node.append("unvisited")
            
            # 4ème key - path
            new_path = l[3].copy() 
            if m[0] not in new_path:
                new_path.append(m[0])
            new_node.append(new_path)
            
            mapper_output.append(new_node)
    else:
        mapper_output.append(l)

mapper_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'unvisited', ['A', 'B']],
 ['C', 4.0, 'unvisited', ['A', 'C']],
 ['D', 8.0, 'unvisited', ['A', 'D']],
 ['B', inf, 'unvisited', ['A'], ['C', 4.0], ['E', 9.0]],
 ['C', inf, 'unvisited', ['A'], ['D', 4.0], ['E', 5.0]],
 ['D', inf, 'unvisited', ['A'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

In [46]:
# Shuffle & sort:

mapper_output = sorted(mapper_output)
mapper_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'unvisited', ['A', 'B']],
 ['B', inf, 'unvisited', ['A'], ['C', 4.0], ['E', 9.0]],
 ['C', 4.0, 'unvisited', ['A', 'C']],
 ['C', inf, 'unvisited', ['A'], ['D', 4.0], ['E', 5.0]],
 ['D', 8.0, 'unvisited', ['A', 'D']],
 ['D', inf, 'unvisited', ['A'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

# A rajouter dans reducer :
juste après la première boucle for 
while line[0] != stop_node: continue


In [48]:
# ITERATION 1: reducer

red_output = []
red_interm = []
key_in_output = []

# on rajoute les valeurs (= adjency nodes) pour chaque list_node qui n'a que la key
for l in mapper_output:
    new_node = l.copy()
    for m in mapper_output:
        if (m[0] == new_node[0]) & (len(m) > 4) & (new_node[1] < m[1]) :
            for i in range(len(m[4:])):
                new_node.append(m[4:][i])
    if len(new_node) > 4:
        red_interm.append(new_node)

# Entre deux list_nodes pour le même node, on ne garde que celle qui a le plus petit weight

for l in red_interm:
    candidate = l # au départ on garde la première
    for m in red_interm:
        if (l[0]== m[0]) & (m[1] < l[1]):
            candidate = m # on parcourt toutes les autres listes, et si une liste est plus petite > nouveau candidat
    
    if candidate[0] not in key_in_output:
        red_output.append(candidate)
        key_in_output.append(candidate[0])


red_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'unvisited', ['A', 'B'], ['C', 4.0], ['E', 9.0]],
 ['C', 4.0, 'unvisited', ['A', 'C'], ['D', 4.0], ['E', 5.0]],
 ['D', 8.0, 'unvisited', ['A', 'D'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

In [49]:
# ITERATION 2: mapper

mapper_output = []

for l in red_output:
    # conditions pour visiter un node: qu'il ne soit pas encore visite et que son weight soit différent de inf
    if (l[2] == "unvisited") & (l[1] != math.inf):
        l[2] = "visited"
        
        # mise à jour du path:
        if l[0] not in l[3]:
            l[3].append(l[0])
        mapper_output.append(l)
        
        for m in l[4:]:
           # nouvelles keys créées à partir des adjancy nodes du node visité
            new_node = m.copy() # on a ici node_name + node distance to last neighbor
            
            # 2nd key - mise à jour des poids associés à chaque adjency node (but = toujours avoir distance from start)
            new_node[1] = new_node[1] + l[1]
            
            # 3ème key - status
            new_node.append("unvisited")
            
            # 4ème key - path
            new_path = l[3].copy() 
            if m[0] not in new_path:
                new_path.append(m[0])
            new_node.append(new_path)
            
            mapper_output.append(new_node)
            
    else:
        mapper_output.append(l)

mapper_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'visited', ['A', 'B'], ['C', 4.0], ['E', 9.0]],
 ['C', 7.0, 'unvisited', ['A', 'B', 'C']],
 ['E', 12.0, 'unvisited', ['A', 'B', 'E']],
 ['C', 4.0, 'visited', ['A', 'C'], ['D', 4.0], ['E', 5.0]],
 ['D', 8.0, 'unvisited', ['A', 'C', 'D']],
 ['E', 9.0, 'unvisited', ['A', 'C', 'E']],
 ['D', 8.0, 'visited', ['A', 'D'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['B', 14.0, 'unvisited', ['A', 'D', 'B']],
 ['E', 15.0, 'unvisited', ['A', 'D', 'E']],
 ['F', 12.0, 'unvisited', ['A', 'D', 'F']],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

In [51]:
# Shuffle & sort:

mapper_output = sorted(mapper_output)
mapper_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'visited', ['A', 'B'], ['C', 4.0], ['E', 9.0]],
 ['B', 14.0, 'unvisited', ['A', 'D', 'B']],
 ['C', 4.0, 'visited', ['A', 'C'], ['D', 4.0], ['E', 5.0]],
 ['C', 7.0, 'unvisited', ['A', 'B', 'C']],
 ['D', 8.0, 'unvisited', ['A', 'C', 'D']],
 ['D', 8.0, 'visited', ['A', 'D'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', 9.0, 'unvisited', ['A', 'C', 'E']],
 ['E', 12.0, 'unvisited', ['A', 'B', 'E']],
 ['E', 15.0, 'unvisited', ['A', 'D', 'E']],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', 12.0, 'unvisited', ['A', 'D', 'F']],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

In [52]:
# ITERATION 2: reducer

red_interm = []
red_output = []
key_in_output = []

# Mise à jour des valeurs
for l in mapper_output:
    new_node = l.copy()
    for m in mapper_output:
        if (m[0] == new_node[0]) & (len(m) > 4) & (new_node[1] < m[1]) :
            for i in range(len(m[4:])):
                new_node.append(m[4:][i])
    if len(new_node) > 4:
        red_interm.append(new_node)


for l in red_interm:
    candidate = l
    for m in red_interm:
        if (l[0]== m[0]) & (m[1] < l[1]):
            candidate = m
    
    if candidate[0] not in key_in_output:
        red_output.append(candidate)
        key_in_output.append(candidate[0])

red_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'visited', ['A', 'B'], ['C', 4.0], ['E', 9.0]],
 ['C', 4.0, 'visited', ['A', 'C'], ['D', 4.0], ['E', 5.0]],
 ['D', 8.0, 'visited', ['A', 'D'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', 9.0, 'unvisited', ['A', 'C', 'E'], ['C', 5.0], ['G', 8.0]],
 ['F', 12.0, 'unvisited', ['A', 'D', 'F'], ['B', 15.0], ['G', 10.0]]]

In [53]:
# ITERATION 3 : mapper

mapper_output = []

for l in red_output:
    new_m = []
    # conditions pour visiter un node: qu'il ne soit pas encore visite et que son weight soit différent de inf
    if (l[2] == "unvisited") & (l[1] != math.inf):
        l[2] = "visited"
        
        # mise à jour du path:
        if l[0] not in l[3]:
            l[3].append(l[0])
        mapper_output.append(l)
        
        for m in l[4:]:
           # nouvelles keys créées à partir des adjancy nodes du node visité
            new_node = m.copy() # on a ici node_name + node distance to last neighbor
            
            # 2nd key - mise à jour des poids associés à chaque adjency node (but = toujours avoir distance from start)
            new_node[1] = new_node[1] + l[1]
            
            # 3ème key - status
            new_node.append("unvisited")
            
            # 4ème key - path
            new_path = l[3].copy() 
            if m[0] not in new_path:
                new_path.append(m[0])
            new_node.append(new_path)
            
            mapper_output.append(new_node)
            
    else:
        mapper_output.append(l)

mapper_output


[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'visited', ['A', 'B'], ['C', 4.0], ['E', 9.0]],
 ['C', 4.0, 'visited', ['A', 'C'], ['D', 4.0], ['E', 5.0]],
 ['D', 8.0, 'visited', ['A', 'D'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', 9.0, 'visited', ['A', 'C', 'E'], ['C', 5.0], ['G', 8.0]],
 ['C', 14.0, 'unvisited', ['A', 'C', 'E']],
 ['G', 17.0, 'unvisited', ['A', 'C', 'E', 'G']],
 ['F', 12.0, 'visited', ['A', 'D', 'F'], ['B', 15.0], ['G', 10.0]],
 ['B', 27.0, 'unvisited', ['A', 'D', 'F', 'B']],
 ['G', 22.0, 'unvisited', ['A', 'D', 'F', 'G']]]

In [54]:
# Shuffle & sort:

mapper_output = sorted(mapper_output)
mapper_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'visited', ['A', 'B'], ['C', 4.0], ['E', 9.0]],
 ['B', 27.0, 'unvisited', ['A', 'D', 'F', 'B']],
 ['C', 4.0, 'visited', ['A', 'C'], ['D', 4.0], ['E', 5.0]],
 ['C', 14.0, 'unvisited', ['A', 'C', 'E']],
 ['D', 8.0, 'visited', ['A', 'D'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', 9.0, 'visited', ['A', 'C', 'E'], ['C', 5.0], ['G', 8.0]],
 ['F', 12.0, 'visited', ['A', 'D', 'F'], ['B', 15.0], ['G', 10.0]],
 ['G', 17.0, 'unvisited', ['A', 'C', 'E', 'G']],
 ['G', 22.0, 'unvisited', ['A', 'D', 'F', 'G']]]

In [55]:
# ITERATION 3: reducer

red_interm = []
red_output = []
key_in_output = []

# Mise à jour des valeurs
for l in mapper_output:
    new_node = l.copy()
    for m in mapper_output:
        if (m[0] == new_node[0]) & (len(m) > 4) & (new_node[1] < m[1]) :
            for i in range(len(m[4:])):
                new_node.append(m[4:][i])
    if len(new_node) > 4:
        red_interm.append(new_node)


for l in red_interm:
    candidate = l
    for m in red_interm:
        if (l[0]== m[0]) & (m[1] < l[1]):
            candidate = m
    
    if candidate[0] not in key_in_output:
        red_output.append(candidate)
        key_in_output.append(candidate[0])

red_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'visited', ['A', 'B'], ['C', 4.0], ['E', 9.0]],
 ['C', 4.0, 'visited', ['A', 'C'], ['D', 4.0], ['E', 5.0]],
 ['D', 8.0, 'visited', ['A', 'D'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', 9.0, 'visited', ['A', 'C', 'E'], ['C', 5.0], ['G', 8.0]],
 ['F', 12.0, 'visited', ['A', 'D', 'F'], ['B', 15.0], ['G', 10.0]]]

# BROUILLON

(au cas où)

In [ ]:
# ITERATION 2: reducer

red_output = []

# Mise à jour des valeurs

for l in mapper_output:
    if len(l) < 5:
        for m in mapper_output:
            if (m[0] == l[0]) & (len(m) > 4):
                for i in range(len(m[4:])):
                    l.append(m[4:][i])

for l in mapper_output:
    new_list = l
    for m in mapper_output:
        if (l[0]== m[0]) & (m[1] < l[1]):
            new_list = m
    
    if new_list not in red_output:
        red_output.append(new_list)

red_output

In [ ]:
red_output = []

# Mise à jour des valeurs

for l in mapper_output:
    if len(mapper_output) < 5:
        for m in mapper_output:
            if (m[0] == l[0]) & (len(m) > 4):
                l.append(m[4:])
        


for i in range(len(mapper_output)):
    current_node = mapper_output[0][0]
    weight_node = mapper_output[0][0]
    
    for l in mapper_output:
        if l[0] == current_node:
            
        
        
        

In [ ]:
# mapper1 BROUILLON

visited_table = []
current_node = start
current_dist = []

for l in adlist:
    if l[1] != math.inf:
        visited_table.append(l[2:])
    else:
        var_list = []
        for j in range(len(l[2:])):
            var_list. append([l[2:][j][0], math.inf])
        current_dist.append(var_list)
            
print(current_dist)
print(visited_table)

In [ ]:
[m[0] for m in visited_table[0]]

In [ ]:
red_output = []

for l in adlist:
    for m in visited_table[0]:
        if l[0] == m[0]:
            l[1] = m[1]
            for i in range(len(l[2:])):
                l[2:][i][1] = l[2:][i][1] + m[1]
                
    red_output.append(l)

red_output

In [ ]:
red_output[2][1] != math.inf

In [ ]:
## MAPPER 2
# on veut avoir la liste des nodes parcourus, et garder les plus petits pour chaque point à partir du node d'avant

step = 2 # on a parcouru tous les nodes qui étaient à 1 vecteur de start, maintenant on veut append les distances
visited_table = []

for l in red_output:
    if l[1] != math.inf:
        for i in range(len(l[2:])):
            visited_table.append(l[2:][i])
        
visited_table

In [ ]:
# ITERATION 1: mapper OLLDDDDDDD

mapper_output = []

for l in adlist:
    new_m = []
    
    # conditions pour visiter un node: qu'il ne soit pas encore visite et que son weight soit différent de inf
    if (l[2] == "unvisited") & (l[1] != math.inf):
        l[2] = "visited"
        
        # mise à jour du path:
        if l[0] not in l[3]:
            l[3].append(l[0])
        mapper_output.append(l)
        
        for m in l[4:]:
            
            # mise à jour des poids associés à chaque adjency node (but = toujours avoir distance from start)
            m[1] = m[1] + l[1]

            # nouvelles keys créées à partir des adjancy nodes du node visité
            new_m = m.copy()
            new_m.append("unvisited")
            new_m.append([l[0]])
            mapper_output.append(new_m)
            m[1] = m[1] + l[1]

    else:
        mapper_output.append(l)

mapper_output